<a href="https://colab.research.google.com/github/FarrazNouval/Python-for-production-and-operation-management/blob/main/material_costing_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Material Costing Using The FIFO Method

Material costing is the process of determining the costs at which inventory items are recorded into stock, as well as their subsequent valuation in the cost accounting records.

There are various approaches or procedures for determining the cost of materials used to manufacture a product in material costing; however, in this notebook, I will be using the FIFO approach.

First in, first out (FIFO) is a material costing method that assumes the costs of the material used for production is the costs of the first material purchased.

The dataset used in this notebook is not a real data, but i tried to make it as similar as possible to the factual dataset for material costing.

steps to do:
1. get the required packages
2. load the dataset
3. check the dataset
4. preprocessing the dataset
5. building the system to solve the problem
6. testing

In [1]:
# get the required packages
import pandas as pd
import os


In [2]:
# load dataframe
df = pd.read_csv("/content/material_costing.csv")
df


,date,status,inventory code,quantity,price,total
0,01/01/2023,in,INV-123456,500,25000.0,12500000.0
1,01/01/2023,in,INV-123457,200,50000.0,10000000.0
2,01/01/2023,in,INV-123458,300,60000.0,18000000.0
3,01/01/2023,in,INV-123459,500,19000.0,9500000.0
4,01/01/2023,in,INV-123460,300,20000.0,6000000.0
5,02/01/2023,in,INV-123457,100,45000.0,4500000.0
6,05/01/2023,out,INV-123458,150,NaN,NaN
7,05/01/2023,out,INV-123460,150,NaN,NaN
8,10/01/2023,out,INV-123456,200,NaN,NaN
9,10/01/2023,out,INV-123457,100,NaN,NaN


In [3]:
# check the dataframe
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            14 non-null     object 
 1   status          14 non-null     object 
 2   inventory code  14 non-null     object 
 3   quantity        14 non-null     int64  
 4   price           7 non-null      float64
 5   total           7 non-null      float64
dtypes: float64(2), int64(1), object(3)
memory usage: 800.0+ bytes


In [4]:
# transform date dtype to datetime
df["date"] = pd.to_datetime(df["date"], format="%d/%m/%Y")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            14 non-null     datetime64[ns]
 1   status          14 non-null     object        
 2   inventory code  14 non-null     object        
 3   quantity        14 non-null     int64         
 4   price           7 non-null      float64       
 5   total           7 non-null      float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 800.0+ bytes


In [5]:
# class for material
class Material:
    def __init__(self, date, inventory, inventory_code, quantity, price):
        self.date = date
        self.inventory = inventory
        self.inventory_code = inventory_code
        self.quantity = quantity
        self.price = price
        self.total = quantity * price

In [6]:
# class for process
class InventoryFIFO:
    def __init__(self, df):
        self.inventory = {}
        self.out_material = {}
        self.df = df

    def filter_data(self, date_from, date_to):
        date_from = pd.to_datetime(date_from, format="%Y/%m/%d")
        date_to = pd.to_datetime(date_to, format="%Y/%m/%d")

        self.df = self.df[(self.df["date"] >= date_from) & (self.df["date"] <= date_to)]

    def add_material(self, material, status):
        if status == "in":
            if material.inventory_code not in self.inventory:
                self.inventory[material.inventory_code] = []
            self.inventory[material.inventory_code].append(material)
        else:
            if material.inventory_code not in self.out_material:
                self.out_material[material.inventory_code] = []
            self.out_material[material.inventory_code].append(material)

    def input_data(self):
        for idx, row in self.df.iterrows():
            if row['status'] == "in":
                self.add_material(Material(row['date'], row['status'], row['inventory code'], row['quantity'], row['price']), "in")
            elif row['status'] == "out":
                cost, minus_quantity = self.remove_material(row['inventory code'], row['quantity'], row['date'])
            elif row['status'] == "return":
                self.return_material(Material(row['date'], row['status'], row['inventory code'], row['quantity'], row['price']))


    def remove_material(self, inventory_code, quantity, date):
        if inventory_code not in self.inventory or not self.inventory[inventory_code]:
            raise ValueError("No inventory available for the given code")

        total_material_quantity = 0
        for i in self.inventory[inventory_code]:
            total_material_quantity += i.quantity

        total_cost = 0
        minus_quantity = 0
        out_quantity = 0
        while quantity > 0 and self.inventory[inventory_code]:
            for i in range(len(self.inventory[inventory_code])):
                current_material = self.inventory[inventory_code][i]
                if total_material_quantity >= quantity:
                    if current_material.quantity < quantity:
                        quantity -= current_material.quantity
                        out_quantity = current_material.quantity
                        current_material.quantity = 0
                        total_cost += current_material.total
                        self.add_material(Material(date,
                                                   "out",
                                                    current_material.inventory_code,
                                                    out_quantity,
                                                    current_material.price), "out")

                    else:
                        current_material.quantity -= quantity
                        total_cost += quantity * current_material.price
                        current_material.total = current_material.quantity * current_material.price
                        out_quantity = quantity
                        quantity = 0
                        self.add_material(Material(date,
                                                   "out",
                                                    current_material.inventory_code,
                                                    out_quantity,
                                                    current_material.price), "out")
                        break
                # jika total material quantity kurang dari quantity(kebutuhan ruang produksi)
                else:
                    if current_material.quantity < quantity:
                        out_quantity += quantity
                        quantity -= current_material.quantity
                        minus_quantity += quantity
                        current_material.quantity = 0
                        total_cost += current_material.total
                        self.add_material(Material(date,
                                                   "out",
                                                    current_material.inventory_code,
                                                    out_quantity,
                                                    current_material.price), "out")


        return total_cost, minus_quantity

    def return_material(self, material):
        current_material = self.inventory[material.inventory_code][-1]
        self.add_material(Material(material.date,
                                   material.inventory,
                                   material.inventory_code, material.quantity,
                                   current_material.price), "in")

    def get_inventory(self, inventory_code):
        return list(self.inventory.get(inventory_code, []))

    def get_removed(self, inventory_code):
        return list(self.out_material.get(inventory_code, []))

    def simplify_material(self, material):
        new_material = {}
        for inv_code in material.keys():
            for inv in material[inv_code]:
                date = inv.date
                status = inv.inventory
                invent_code = inv.inventory_code
                qty = inv.quantity
                price = inv.price
                total = inv.total

                data = [date, status, invent_code, qty, price, total]

                if inv_code not in new_material.keys():
                    new_material[inv_code] = []
                    new_material[inv_code].append(data)
                else:
                    new_material[inv_code].append(data)

        return new_material

    def to_material_df(self, simplified_material, show_zeros: bool):
        new_material = {"date":[],
                        "status":[],
                        "inventory code":[],
                        "quantity":[],
                        "price":[],
                        "total":[]}

        for inv_code in simplified_material.keys():
            for item in simplified_material[inv_code]:
                new_material["date"].append(item[0])
                new_material["status"].append(item[1])
                new_material["inventory code"].append(item[2])
                new_material["quantity"].append(item[3])
                new_material["price"].append(item[4])
                new_material["total"].append(item[5])

        material_df = pd.DataFrame(new_material)
        material_df.sort_values(by="inventory code", inplace=True)
        material_df.reset_index(inplace=True)
        del material_df['index']

        if not show_zeros:
            material_df = material_df[material_df["quantity"] > 0]

        return material_df


In [7]:
##############################################################################################################################
# make an object of inventory
inv_fifo = InventoryFIFO(df)

print("enter date to filter")
date_from = input("start date, format = yyyy/mm/dd [2022/07/22]")
date_to = input("end date, format = yyyy/mm/dd [2022/07/22]")
show_zeros = input("do you want to show material with zero quantity or not, Y/N")

inv_fifo.filter_data(date_from, date_to)
inv_fifo.input_data()

remaining_inventory = inv_fifo.inventory
out_inventory = inv_fifo.out_material

simple_remaining_material = inv_fifo.simplify_material(remaining_inventory)
simple_out_material = inv_fifo.simplify_material(out_inventory)

if show_zeros.lower() == "y":
    remaining_material_df = inv_fifo.to_material_df(simple_remaining_material, True)
    out_material_df = inv_fifo.to_material_df(simple_out_material, True)
else:
    remaining_material_df = inv_fifo.to_material_df(simple_remaining_material, False)
    out_material_df = inv_fifo.to_material_df(simple_out_material, False)


enter date to filter
start date, format = yyyy/mm/dd [2022/07/22]2023/01/01
end date, format = yyyy/mm/dd [2022/07/22]2023/01/20
do you want to show material with zero quantity or not, Y/NY


In [10]:
remaining_material_df


,date,status,inventory code,quantity,price,total
0,2023-01-01,in,INV-123456,300,25000.0,7500000.0
1,2023-01-01,in,INV-123457,0,50000.0,5000000.0
2,2023-01-02,in,INV-123457,50,45000.0,2250000.0
3,2023-01-15,in,INV-123457,200,45500.0,9100000.0
4,2023-01-20,return,INV-123457,100,45500.0,4550000.0
5,2023-01-01,in,INV-123458,150,60000.0,9000000.0
6,2023-01-01,in,INV-123459,300,19000.0,5700000.0
7,2023-01-01,in,INV-123460,150,20000.0,3000000.0


In [11]:
out_material_df


,date,status,inventory code,quantity,price,total
0,2023-01-10,out,INV-123456,200,25000.0,5000000.0
1,2023-01-10,out,INV-123457,100,50000.0,5000000.0
2,2023-01-11,out,INV-123457,100,50000.0,5000000.0
3,2023-01-11,out,INV-123457,50,45000.0,2250000.0
4,2023-01-05,out,INV-123458,150,60000.0,9000000.0
5,2023-01-15,out,INV-123459,200,19000.0,3800000.0
6,2023-01-05,out,INV-123460,150,20000.0,3000000.0


In [12]:
# total cost
total_material_cost = out_material_df.total.sum()
total_material_cost


33050000.0

In [13]:
# total remaining inventory
total_remaining_material = remaining_material_df.total.sum()
total_remaining_material


46100000.0

assuming the company produces 150000 units of product x from 2020/01/01 to 2020/01/20, how much is the unit cost of product x?

In [17]:
unit_produced = 150000
unit_cost = round(total_material_cost / unit_produced, 2)
print(f"unit cost : {unit_cost}")


unit cost : 220.33


Based on the analysis, we can conclude that:
1. The total material cost to produce product x from 2020/01/01 to 2020/01/20 is 33.050.000.
2. The total remaining material values in the inventory at 2020/01/20 after production are 46.100.000.
3. The unit cost of product x is 220.33.

This data can be used to evaluate the manufacturing process, assess the effectiveness of the production plan, and establish the efficiency of the manufacturing process.

Furthermore, the company can optimize its production process to reduce costs while increasing profits.